In [1]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()
import geemap


In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles
from models.ResNet import ResNet101

In [3]:
# Adds Earth Engine dataset
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")

fuseImage = ee.ImageCollection('users/410093033/yizhixing').min().rename("fuseImage")
forest23 = fuseImage.expression("b(0)>=2&&b(0)<=3").rename("forest23")

# add landsat80------------------------------------------------------------------------------------
Landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
LC = Landsat.filter(ee.Filter.calendarRange(2020, 2020, 'year')).select(selVar).map(utils.maskL8sr)
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
LC = LC.select(selVar).median()

ndvi = LC.normalizedDifference(['B5', 'B4']).rename('NDVI')
ndwi = LC.normalizedDifference(['B3', 'B5']).rename('NDWI')
DEM = ee.Image("MERIT/DEM/v1_0_3")
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
stratified_classes = ndvi.expression('(b(0)/0.2)').int().rename('STRATIFIED_CLASSES')
LC_STN = LC.addBands(ndvi).addBands(ndwi).addBands(slope).addBands(stratified_classes).float()

selVar1 = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','slope'])


In [4]:
path_fuseImage = r"./fused_01.tif"
path_ge = r"./ge_17.tif"

padding = 0.002

xmin = 111.0
ymin = 30.0
xmax = 111.1
ymax = 30.1

rect = ee.Algorithms.GeometryConstructors.Rectangle(ee.List([xmin, ymin, xmax, ymax]))

In [5]:
if not os.path.exists(path_ge):
    GetMapTiles.getpic_tif(xmin-padding, ymax+padding, xmax+padding, ymin-padding, 17, source='google', out_filename=path_ge, style='s')
if not os.path.exists(path_fuseImage):
    geemap.ee_export_image(fuseImage, path_fuseImage, scale=30, region=rect)

In [6]:
llList_full = common.getLonLatListFromImage(path_fuseImage)
idx_unsure = np.where((llList_full[:, 2]==2) + (llList_full[:, 2]==3))[0]
idx_sure_pos = np.where((llList_full[:, 2]==4) + (llList_full[:, 2]==5))[0]
idx_sure_neg = np.where((llList_full[:, 2]==0) + (llList_full[:, 2]==1))[0]

llList_unsure = llList_full[idx_unsure]
llList_sure_pos = llList_full[idx_sure_pos]
llList_sure_neg = llList_full[idx_sure_neg]

In [8]:
dataLoader = dataset.getLonLatDataLoader(llList_unsure[:, 0:2], local=True, imgSavePath=path_ge, BSize=16, nWorkers=0, pinMem=True, APIKEY=None)
model = ResNet101(in_ch=3, n_classes=5)
pred, offset = common.inference(model, data_loader=dataLoader, device=torch.device('cuda:0'), device_ids=(0,), comment="ResNet101_GE17_1206", save_path="./CKPT/")
predforest = (pred.cpu().numpy()==0).astype(np.int)

100%|██████████| 859/859 [01:13<00:00, 11.75it/s]


In [29]:
# get all sample as 0-1 list
llList_forest_sure_pos = llList_sure_pos.copy()
llList_forest_sure_neg = llList_sure_neg.copy()
llList_forest_unsure = llList_unsure.copy()

llList_forest_sure_pos[:, 2] = (llList_sure_pos[:, 2]>=4).astype(np.int)
llList_forest_sure_neg[:, 2] = (llList_sure_neg[:, 2]>=4).astype(np.int)
llList_forest_unsure[:, 2] = predforest

# apply offset to llList_forest_unsure
llList_forest_unsure_offseted = llList_forest_unsure.copy()
llList_forest_unsure_offseted[:, [0, 1]] += offset.cpu().numpy()

# select equal samples from sure_pos, sure_neg from sure area
sampleNum = 4000
llList_forest_sure_pos_sample = random.sample(llList_forest_sure_pos.tolist(), sampleNum)
llList_forest_sure_neg_sample = random.sample(llList_forest_sure_neg.tolist(), sampleNum)

# select samples from unsure area
unsure_sample_idx = random.sample(list(range(0, llList_forest_unsure.shape[0])), 4000)
llList_forest_unsure_sample = llList_forest_unsure[unsure_sample_idx].tolist()
llList_forest_unsure_offseted_sample = llList_forest_unsure_offseted[unsure_sample_idx].tolist()

In [30]:
rect_05 = ee.Algorithms.GeometryConstructors.Rectangle(ee.List([xmin-0.2, ymin-0.2, xmax+0.3, ymax+0.3]))

llList_forest_sample = llList_forest_sure_pos_sample + llList_forest_sure_neg_sample + llList_forest_unsure_sample
llList_forest_sample_offseted = llList_forest_sure_pos_sample + llList_forest_sure_neg_sample + llList_forest_unsure_offseted_sample

# pred with original samples
CrdClsList = ee.List(llList_forest_sample)
mapped = CrdClsList.map(lambda x : ee.Feature(ee.Algorithms.GeometryConstructors.Point(ee.List(x).slice(0, 2)), {"forest":ee.List(x).get(2)}))
fc_points = ee.FeatureCollection(mapped)
sample_train = LC_STN.sampleRegions(collection=fc_points, geometries=True, scale=30)
classifier = ee.Classifier.smileRandomForest(numberOfTrees=200, variablesPerSplit=9, minLeafPopulation=1, bagFraction=0.5, maxNodes=None, seed=0).train(sample_train, "forest", selVar1)
predForest = LC_STN.clip(rect_05).select(selVar1).classify(classifier)

# pred with offseted samples
CrdClsList_offseted = ee.List(llList_forest_sample_offseted)
mapped = CrdClsList_offseted.map(lambda x : ee.Feature(ee.Algorithms.GeometryConstructors.Point(ee.List(x).slice(0, 2)), {"forest":ee.List(x).get(2)}))
fc_points_offseted = ee.FeatureCollection(mapped)
sample_train_offseted = LC_STN.sampleRegions(collection=fc_points_offseted, geometries=True, scale=30)
classifier = ee.Classifier.smileRandomForest(numberOfTrees=200, variablesPerSplit=9, minLeafPopulation=1, bagFraction=0.5, maxNodes=None, seed=0).train(sample_train_offseted, "forest", selVar1)
predForest_offseted = LC_STN.clip(rect_05).select(selVar1).classify(classifier)

In [ ]:
Map = geemap.Map(center=[30, 111], zoom=10, add_google_map=True)


Map.addLayer(LC_STN.clip(rect_05), vis_params={"max":2000, "min":0, "bands":['B4', 'B3', 'B2']}, name="LC08")
Map.addLayer(predForest, vis_params={"max":1, "min":0, "palette":["FF0000", "00FF00"]}, name="pred", opacity=1)
Map.addLayer(predForest_offseted, vis_params={"max":1, "min":0, "palette":["FF0000", "00FF00"]}, name="pred_offseted", opacity=1)
Map.addLayer(fuseImage.clip(rect_05), vis_params={"max":5, "min":0, "palette":["FF0000", "880000", "FF9900", "FFFF00", "00FF00", "00FF00"]}, name="fuseImage", opacity=1)

# original sample
Map.addLayer(fc_points.filterMetadata("forest", 'equals', 1).draw(color='00FF00', pointRadius=1), name="sample_pos")
Map.addLayer(fc_points.filterMetadata("forest", 'equals', 0).draw(color='FF0000', pointRadius=1), name="sample_neg")

# offseted sample
Map.addLayer(fc_points_offseted.filterMetadata("forest", 'equals', 1).draw(color='00FFAA', pointRadius=1), name="sample_pos_offseted")
Map.addLayer(fc_points_offseted.filterMetadata("forest", 'equals', 0).draw(color='FFAA00', pointRadius=1), name="sample_neg_offseted")

Map.add_basemap('Google Satellite')
Map.addLayerControl()
Map